In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#import basic libraries
import pandas as pd
import string
import numpy as np
import json

In [3]:
#import required kerals modules
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ut

In [4]:
#import required tf modules
import tensorflow as tf

In [5]:
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

In [6]:
#load all datasets
ds1 = pd.read_csv('/content/gdrive/MyDrive/Data/USvideos.csv')
ds2 = pd.read_csv('/content/gdrive/MyDrive/Data/CAvideos.csv')
ds3 = pd.read_csv('/content/gdrive/MyDrive/Data/GBvideos.csv')

In [7]:
ds1.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [8]:
ds2.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...


In [9]:
ds3.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Jw1Y-zhQURU,17.14.11,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,26,2017-11-10T07:38:29.000Z,"christmas|""john lewis christmas""|""john lewis""|...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...
1,3s1rvMFUweQ,17.14.11,Taylor Swift: …Ready for It? (Live) - SNL,Saturday Night Live,24,2017-11-12T06:24:44.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs …Ready for...
2,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
3,PUTEiSjKwJU,17.14.11,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,17,2017-11-13T02:30:38.000Z,"Salford City FC|""Salford City""|""Salford""|""Clas...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...
4,rHwDegptbI4,17.14.11,Dashcam captures truck's near miss with child ...,Cute Girl Videos,25,2017-11-13T01:45:13.000Z,[none],9815,30,2,30,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,False,False,False,Dashcam captures truck's near miss with child ...


In [10]:
#load the json data
js1 = json.load(open('/content/gdrive/MyDrive/Data/US_category_id.json'))
js2 = json.load(open('/content/gdrive/MyDrive/Data/CA_category_id.json'))
js3 = json.load(open('/content/gdrive/MyDrive/Data/GB_category_id.json'))

In [11]:
js1['items']

[{'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
  'id': '1',
  'kind': 'youtube#videoCategory',
  'snippet': {'assignable': True,
   'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
   'title': 'Film & Animation'}},
 {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"',
  'id': '2',
  'kind': 'youtube#videoCategory',
  'snippet': {'assignable': True,
   'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
   'title': 'Autos & Vehicles'}},
 {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
  'id': '10',
  'kind': 'youtube#videoCategory',
  'snippet': {'assignable': True,
   'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
   'title': 'Music'}},
 {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-oBJavSGkfDI"',
  'id': '15',
  'kind': 'youtube#videoCategory',
  'snippet': {'assignable': True,
   'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
   'title': 'Pets & Animals'}},
 {'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM1XVQ9zbGec"',
  'id': '17',
  

In [12]:
#extracting categorys from json data
def category_extractor(data):
  i_d=[data['items'][i]['id'] for i in range(len(data['items']))]
  title = [data['items'][i]['snippet']["title"] for i in range(len(data['items']))]
  i_d=list(map(int,i_d))
  category=zip(i_d,title)
  category=dict(category)
  return category

In [13]:
#creating new column in ds data files
ds1['category_title']=ds1['category_id'].map(category_extractor(js1))
ds2['category_title']=ds2['category_id'].map(category_extractor(js2))
ds3['category_title']=ds3['category_id'].map(category_extractor(js3))

In [14]:
ds1.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_title
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,People & Blogs
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",Entertainment
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,Comedy
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,Entertainment
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,Entertainment


In [15]:
#now lets join the all ds data files
dataset=pd.concat([ds1,ds2,ds3],ignore_index=True)

In [16]:
len(dataset)

120746

In [17]:
#now let's drop the duplicate videos based on video id
dataset=dataset.drop_duplicates('video_id')

In [18]:
len(dataset)

30318

In [22]:
dataset['category_title'].unique()

array(['People & Blogs', 'Entertainment', 'Comedy',
       'Science & Technology', 'Film & Animation', 'News & Politics',
       'Sports', 'Music', 'Pets & Animals', 'Education', 'Howto & Style',
       'Autos & Vehicles', 'Travel & Events', 'Gaming',
       'Nonprofits & Activism', 'Shows', nan, 'Movies'], dtype=object)

In [43]:
#let's choose entertainment category
ct_name='Entertainment'
ct_category=dataset[dataset['category_title']==ct_name]['title']

In [44]:
len(ct_category)

9730

In [45]:
ct_category.head()

1     The Trump Presidency: Last Week Tonight with J...
3                      Nickelback Lyrics: Real or Fake?
4                              I Dare You: GOING BALD!?
6             Roy Moore & Jeff Sessions Cold Open - SNL
11    (SPOILERS) 'Shiva Saves the Day' Talked About ...
Name: title, dtype: object

In [46]:
ct_category = ct_category.tolist()

In [47]:
#now let's remove punctuation and convert text to lowercase
def clean_text(text):
  text=''.join(e for e in text if e not in string.punctuation).lower()
  text = text.encode('utf8').decode('ascii', 'ignore')
  return text

In [48]:
corpus=[clean_text(e) for e in ct_category]

In [49]:
corpus[:5]

['the trump presidency last week tonight with john oliver hbo',
 'nickelback lyrics real or fake',
 'i dare you going bald',
 'roy moore  jeff sessions cold open  snl',
 'spoilers shiva saves the day talked about scene ep 804  the walking dead']

In [50]:
tokenizer=Tokenizer()

In [51]:
def get_sequence_of_tokens(corpus):
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index) + 1
  
  #convert to sequence of tokens
  input_sequences=[]
  for line in corpus:
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
      n_gram_sequence=token_list[:i+1]
      input_sequences.append(n_gram_sequence)
  return input_sequences,total_words

In [52]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)

In [53]:
total_words

13915

In [54]:
def generate_padding_sequences(input_sequences):
  max_sequence_len = max([len(x) for x in input_sequences])
  input_sequences = np.array(pad_sequences(input_sequences,  maxlen=max_sequence_len, padding='pre'))
  predictors, label = input_sequences[:,:-1], input_sequences[:, -1]
  label = ut.to_categorical(label, num_classes = total_words)
  return predictors, label, max_sequence_len

In [55]:
predictors, label, max_sequence_len = generate_padding_sequences(inp_sequences)

In [56]:
#create lstm model
def create_model(max_seq_length,total_words):
  input_len=max_sequence_len-1
  model=Sequential()
  model.add(Embedding(total_words,10,input_length=input_len))
  model.add(LSTM(100))
  model.add(Dropout(0.1))
  model.add(Dense(total_words, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')
  return model

In [57]:
model=create_model(max_sequence_len,total_words)

In [58]:
model.fit(predictors, label, epochs=100)

Epoch 1/100
2202/2202 [==============================] - 17s 7ms/step - loss: 8.1983
Epoch 2/100
2202/2202 [==============================] - 16s 7ms/step - loss: 7.1825
Epoch 3/100
2202/2202 [==============================] - 16s 7ms/step - loss: 6.6457
Epoch 4/100
2202/2202 [==============================] - 16s 7ms/step - loss: 6.1994
Epoch 5/100
2202/2202 [==============================] - 16s 7ms/step - loss: 5.8506
Epoch 6/100
2202/2202 [==============================] - 16s 7ms/step - loss: 5.4778
Epoch 7/100
2202/2202 [==============================] - 16s 7ms/step - loss: 5.1233
Epoch 8/100
2202/2202 [==============================] - 15s 7ms/step - loss: 4.8367
Epoch 9/100
2202/2202 [==============================] - 16s 7ms/step - loss: 4.5688
Epoch 10/100
2202/2202 [==============================] - 16s 7ms/step - loss: 4.3286
Epoch 11/100
2202/2202 [==============================] - 16s 7ms/step - loss: 4.1085
Epoch 12/100
2202/2202 [==============================] - 16s 7

In [64]:
def generate_text(seed_text, next_words, model, max_sequence_len):
  for i in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1,  padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
 
    output_word = " "
    for word,index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " "+output_word
  return seed_text.title()

In [70]:
print(generate_text("avengers Fight", 5, model, max_sequence_len))

Avengers Fight Ep 1 April 2018 Promo


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
